### Ejemplo de uso de tensorflow para predecir el dataset Iris

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import normalize

##### Lectura de datos
1. Carga de datos.
2. Codificación de target y.

In [2]:
data = datasets.load_iris()
labelBinarizer = LabelBinarizer()

dataset = pd.DataFrame(data.data, columns=['sepal_length','sepal_width','petal_length','petal_width'])
dataset['specie'] = data.target
Y = labelBinarizer.fit_transform(dataset.specie.values)

#### Normalización de los datos de entrada

In [3]:
X = dataset.columns[0:4]
X_data = dataset[X].values
X_data = normalize(X_data)


#### Dividir los datos en entrenamiento y test

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_data, Y, random_state = 1)

#### Construcción de la red neural con tensorflow

La red neuronal propuesta tendrá dos capas de 256 y 128 neuronas respectivamente y una capa de salida.

In [12]:
learning_rate = 0.005
training_epochs = 3000
n_hidden1 = 25  # Number of neurons in layer 1
#n_hidden2 = 128 # Number of neurons in layer 2
n_input = X_train.shape[1] #Number of features (4)
n_classes = y_train.shape[1] #Number of classes to predict


In [13]:
X = tf.placeholder(shape=[None,n_input], dtype = tf.float32)
y = tf.placeholder(shape=[None,n_classes], dtype = tf.float32)

#Weights
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
    'out': tf.Variable(tf.random_normal([n_hidden1, n_classes]))
}

#Biases
biases = {
  'b1': tf.Variable(tf.random_normal([n_hidden1])),
  'out': tf.Variable(tf.random_normal([n_classes]))
}

#### Forward propagation

In [14]:
def forward_propagation(x):
    #Hidden layer 1.
    out_h1 = tf.nn.relu(tf.matmul(x,weights['h1'])+biases['b1'])    
    #Output layer
    out_layer = tf.matmul(out_h1, weights['out'])+biases['out']
    
    return out_layer

y_hat = forward_propagation(X)
y_predict = tf.argmax(y_hat, axis = 1) #Devuelve la columna con el mayor valor.
    


#### Backward propagation

In [15]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits =y_hat))
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_op= optimizer.minimize(cost)

#### Entrenamiento de la red neuronal

In [16]:
    
    arrays = np.array([[0.71066905, 0.35533453, 0.56853524, 0.21320072],
       [0.72415258, 0.32534391, 0.56672811, 0.22039426],
       [0.69997037, 0.32386689, 0.58504986, 0.25073566],
       [0.73337886, 0.32948905, 0.54206264, 0.24445962],
       [0.69052512, 0.32145135, 0.60718588, 0.22620651],
       [0.69193502, 0.32561648, 0.60035539, 0.23403685],
       [0.68914871, 0.33943145, 0.58629069, 0.25714504],
       [0.72155725, 0.32308533, 0.56001458, 0.24769876],
       [0.72965359, 0.28954508, 0.57909015, 0.22005426],
       [0.71653899, 0.3307103 , 0.57323119, 0.22047353],
       [0.67467072, 0.36998072, 0.58761643, 0.25028107],
       [0.69025916, 0.35097923, 0.5966647 , 0.21058754]])
    
    print(arrays)

[[0.71066905 0.35533453 0.56853524 0.21320072]
 [0.72415258 0.32534391 0.56672811 0.22039426]
 [0.69997037 0.32386689 0.58504986 0.25073566]
 [0.73337886 0.32948905 0.54206264 0.24445962]
 [0.69052512 0.32145135 0.60718588 0.22620651]
 [0.69193502 0.32561648 0.60035539 0.23403685]
 [0.68914871 0.33943145 0.58629069 0.25714504]
 [0.72155725 0.32308533 0.56001458 0.24769876]
 [0.72965359 0.28954508 0.57909015 0.22005426]
 [0.71653899 0.3307103  0.57323119 0.22047353]
 [0.67467072 0.36998072 0.58761643 0.25028107]
 [0.69025916 0.35097923 0.5966647  0.21058754]]


In [17]:
from datetime import datetime
import matplotlib.pyplot as plt
import time

# Initializing the variables
init = tf.global_variables_initializer()
batch_size = 120
loss = []
start = time.time()
with tf.Session() as sess:
    sess.run(init)
    
    #writer.add_graph(sess.graph)
    #EPOCHS
    for epoch in range(training_epochs):
        size = np.random.choice(len(X_train),batch_size)

        x_batch = X_train[size]
        y_batch = y_train[size]
        #Stochasting Gradient Descent
        #for i in range(len(X_train)):
        #    summary = sess.run(train_op, feed_dict={X: X_train[i:i+1], y: y_train[i:i+1]})
        #for i in range(len(X_train)):
        summary = sess.run(train_op, feed_dict={X: x_batch, y: y_batch})
        
        train_accuracy = np.mean(np.argmax(y_train, axis=1) == sess.run(y_predict, feed_dict={X: X_train, y: y_train}))
        test_accuracy  = np.mean(np.argmax(y_test, axis=1) == sess.run(y_predict, feed_dict={X: X_test, y: y_test}))
        loss.append(sess.run(cost, feed_dict={X: X_train, y: y_train}))
        if ((epoch +1 ) % 1000 == 0):
            print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%" % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy))

    
   # print(sess.run(forward_propagation([[1.,1.,1.,1.]])))
    for array in arrays:
        print(sess.run(tf.argmax(forward_propagation(np.float32(array.reshape(1,4))),axis=1)))
    sess.close()

    end = time.time()
    
    print("Total time: {0}".format(end - start))

Epoch = 1000, train accuracy = 76.79%, test accuracy = 78.95%
Epoch = 2000, train accuracy = 89.29%, test accuracy = 84.21%
Epoch = 3000, train accuracy = 95.54%, test accuracy = 86.84%
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
Total time: 7.674698114395142


In [ ]:
plt.plot(np.array(range(1,5001)),loss)

In [ ]:
plt.plot(np.array(range(1,201)),loss)

In [ ]:
session = tf.Session()
print(forward_propagation([[1.,1.,1.,1.]]))
#session.run(forward_propagation([[1.,1.,1.,1.]]))